Canada Covid-19 Case Study

In [ ]:
import pandas as pd
import numpy as np

wk12virus_df = pd.read_csv('Canada Coronavirus Wk12.csv')
wk13virus_df = pd.read_csv('Canada Coronavirus Wk13.csv')

wk12orders_df = pd.read_csv('Canada Orders Wk12.csv')
wk13orders_df = pd.read_csv('Canada Orders Wk13.csv')

Creating DataFrames for each file

In [ ]:
# dataframe for week 12 virus file
wk12virus_df.head(10)

In [ ]:
# dataframe for week 13 virus file
wk13virus_df.head()

In [ ]:
# dataframe for week 12 order file
wk12orders_df.head()
wk12orders_df['POSTAL_FIRST_LETTER'] = wk12orders_df['SHIP_TO_POSTAL_CODE'].str.slice(0, 1)
wk12orders_df.groupby('POSTAL_FIRST_LETTER').agg({'CARTON_ESTIMATION_COUNT':"sum"})

In [ ]:
# dataframe for week 13 order file
wk13orders_df.head()
wk13orders_df['POSTAL_FIRST_LETTER'] = wk13orders_df['SHIP_TO_POSTAL_CODE'].str.slice(0, 1)
wk13orders_df.groupby(['POSTAL_FIRST_LETTER']).agg({'CARTON_ESTIMATION_COUNT':"sum"})

Cleaning up week 13 orders data

In [ ]:
# Clean up week 13 by fixing TBD and replacing with a space
wk13orders_df['SHIP_TO_STATE'] = wk13orders_df['SHIP_TO_STATE'].replace('TBD','')

In [ ]:
wk13orders_df.head()

Creating a Covid Impact DataFrame along with an Order DataFrame

In [ ]:
#Creating a covid impact data frame, and making it readable
#outer join to ensure no data is lost
covid_impact_df = pd.merge(wk12virus_df, wk13virus_df, how = 'outer', left_on=['Provinence', 'Postal Code First Letter'], right_on = ['Provinence', 'Postal Code First Letter'])

# dropped column because its information is elsewhere
covid_impact_df.drop(['Country_y'], axis=1, inplace =True)

# renaming columns
covid_impact_df.rename(columns = {'Week_x':'Week 12', 'Weekly Cases_x':'Weekly Cases Wk 12', 
                                  'Weekly Impact_x':'Weekly Impact Wk 12','Country_x':'Country',
                                  'Total Cases_x': 'Total Cases Wk 12','Overall Impact_x':'Overall Impact Wk 12' 
                                 }, inplace = True)

covid_impact_df.rename(columns = {'Week_y':'Week 13', 'Weekly Cases_y':'Weekly Cases Wk 13', 
                                  'Weekly Impact_y':'Weekly Impact Wk 13',
                                  'Total Cases_y': 'Total Cases Wk 13','Overall Impact_y':'Overall Impact Wk 13' 
                                 }, inplace = True)

covid_impact_df.head(20)

In [ ]:
# table made for the order data, outer join again to make sure no data lost
order_df = pd.merge(wk12orders_df, wk13orders_df, how = 'outer', left_on=['ORDER_NUMBER', 'SHIP_TO_POSTAL_CODE'], right_on =['ORDER_NUMBER', 'SHIP_TO_POSTAL_CODE'])
order_df['POSTAL_FIRST_LETTER'] = order_df['SHIP_TO_POSTAL_CODE'].str.slice(0, 1)

order_df.rename(columns = {'SHIP_TO_CITY_x' : 'SHIP_TO_CITY_Wk12','SHIP_TO_COUNTRY_x':'Country'}, inplace = True)
order_df.rename(columns = {'SHIP_TO_STATE_x' : 'SHIP_TO_STATE_Wk12','SOL_REVENUE_x':'SOL_REVENUE_Wk12'}, inplace = True)
order_df.rename(columns = {'CARTON_ESTIMATION_COUNT_x' : 'CARTON_ESTIMATION_COUNT_Wk12','SALES_HIERARCHY_LEVEL4_x':'SALES_HIERARCHY_LEVEL4_Wk12'}, inplace = True)

order_df.rename(columns = {'SHIP_TO_CITY_y' : 'SHIP_TO_CITY_Wk13'}, inplace = True)
order_df.rename(columns = {'SHIP_TO_STATE_y' : 'SHIP_TO_STATE_Wk13','SOL_REVENUE_y':'SOL_REVENUE_Wk13'}, inplace = True)
order_df.rename(columns = {'CARTON_ESTIMATION_COUNT_y' : 'CARTON_ESTIMATION_COUNT_Wk13','SALES_HIERARCHY_LEVEL4_y':'SALES_HIERARCHY_LEVEL4_Wk13'}, inplace = True)

order_df.drop(['SHIP_TO_COUNTRY_y'], axis=1, inplace =True)

Merging both Order DataFrame and the Covid Impact DataFrame to create a Combined DataFrame

In [ ]:
# combined Merged Data Frame of both
combined_df = pd.merge(covid_impact_df, order_df, left_on = 'Postal Code First Letter', right_on = 'POSTAL_FIRST_LETTER')
#dropping duplicates 
combined_df.drop_duplicates(inplace = True)

combined_df.head(50)

Finding out the most impacted provinence for Week 12 (Most impacted meaning most cases)

In [ ]:
#finding out the most impacted provinence by wk12 weekly case count 
max_values = wk12virus_df['Weekly Cases']
max_value.max()

In [ ]:
wk12max = wk12virus_df

In [ ]:
wk12max.drop(['Postal Code First Letter'], axis=1, inplace =True)

In [ ]:
wk12max.drop_duplicates(inplace = True)
# Basically, going to sort so that the top of the dataframe will have the most weekly cases and then going to use iloc[0] to get the top provinence value
wk12max.sort_values(by=['Weekly Cases'], ascending = False, inplace = True)
maxProvinence = wk12max['Provinence'].iloc[0]
wk12max.head(15)

In [ ]:
maxProvinence = wk12max['Provinence'].iloc[0]
maxProvinence 

Finding out the least impacted provinence for Week 13 (Lease amount of cases for least impact)

In [ ]:
#finding out the least impacted provinence by wk13 weekly case count 

# Creating new dataframe
wk13min = wk13virus_df

#dropped postal code column so that duplicated wil be dropped (do not need it, but made copy so it will not impact original data frame)
wk13min.drop(['Postal Code First Letter'], axis=1, inplace =True)
wk13min.drop_duplicates(inplace = True)

# ascending true makes it go from smallest -> largest weekly cases from top row
wk13min.sort_values(by=['Weekly Cases'], ascending = True, inplace = True)
maxProvinence = wk13min['Provinence'].iloc[0]
wk13min.head(15)

In [ ]:
#similar method to what was used to get most impacted
minProvinence = wk13min['Provinence'].iloc[0]
minProvinence

Finding the total impacted revenue for week 12 for each sales organization

In [ ]:
#total impacted revenue by sales organization
wk12orders_df.head()

In [ ]:
wk12revenue = wk12orders_df.groupby(['SALES_HIERARCHY_LEVEL4'], as_index = False).agg({'SOL_REVENUE':"sum"})
#(df.groupby("Groups", as_index=False).agg({"Date":"first", "data1":"sum", "data2":"sum"}))

In [ ]:
wk12revenue.head()

Within the most impacted provinence, find the total impacted revenue for each sales organziation in week 13

In [ ]:
#Find out the total impacted revenue in week13 for each sales organization within the most impacted provenience (most impacted by case count)
wk13max = wk13virus_df
#wk13max.drop(['Postal Code First Letter'], axis=1, inplace =True)
wk13max.drop_duplicates(inplace = True)
wk13max.sort_values(by=['Weekly Cases'], ascending = False, inplace = True)

mostAffectedProv = wk13max['Provinence'].iloc[0]

mostAffectedProv

In [ ]:
wk13affectedOrders = combined_df
wk13affectedOrders = combined_df.groupby(['SALES_HIERARCHY_LEVEL4_Wk13','Provinence'], as_index = False).agg({'SOL_REVENUE_Wk13':"sum"})
filter = wk13affectedOrders['Provinence'] == mostAffectedProv


In [ ]:
wk13affectedOrders.head(25)

In [ ]:
wk13affectedOrders[filter]

During Week 12, how many cartons will be stuck within each postal code

In [ ]:
#How many cartons will be stuck within each postal code(use column: Postal Code FirstLetter)during wk12?

wk12orders_df['POSTAL_FIRST_LETTER'] = wk12orders_df['SHIP_TO_POSTAL_CODE'].str.slice(0, 1)
wk12carton = wk12orders_df
wk12carton = wk12orders_df.groupby(['POSTAL_FIRST_LETTER'], as_index = False).agg({'CARTON_ESTIMATION_COUNT':"sum"})


In [ ]:
wk12carton.head(30)

Finding how many cartons will be stuck in Postal Code First Letter “X” by provenience “Northwest Territories” and “Nunavut” during wk13

In [ ]:
#stuckCartons = combined_df.where(combined_df['Postal Code First Letter'] == 'L').groupby(['Provinence'], as_index=False).agg({'CARTON_ESTIMATION_COUNT_Wk12':"sum"})

wk13virus_df = pd.read_csv('Canada Coronavirus Wk13.csv')
dfCarton = pd.merge(wk13orders_df,wk13virus_df, how = 'left', left_on='POSTAL_FIRST_LETTER',right_on='Postal Code First Letter')

dfCartonstuck = dfCarton.where(dfCarton['Postal Code First Letter'] == 'L').groupby(['Provinence'], as_index=False).agg({'CARTON_ESTIMATION_COUNT':"sum"})
dfCartonstuck